In [ ]:
import sqlite3
import nltk
from nltk import word_tokenize
from fuzzywuzzy import process
from collections import Counter
import pandas as pd

In [ ]:
tags_df = pd.read_csv('./tag_names_stack_overflow.csv') 
allowable_tokens = tags_df['tagName'].str.lower().tolist()

# Connect to the SQLite database and load job descriptions
conn = sqlite3.connect('your_database.db')
cursor = conn.cursor()

# Fetch all job descriptions
cursor.execute("SELECT job_description FROM job_descriptions")
job_descriptions = cursor.fetchall()

# Combine all job descriptions into a single string for processing
text = " ".join([desc[0] for desc in job_descriptions])

tokens = word_tokenize(text)
tokens = [word.lower() for word in tokens if word.isalpha()]

# Define a function to filter tokens based on allowable tokens using fuzzy matching
def filter_tokens(tokens, allowable_tokens, threshold=80):
    filtered_tokens = []
    for token in tokens:
        match, score = process.extractOne(token, allowable_tokens)
        if score >= threshold:
            filtered_tokens.append(match)
    return filtered_tokens

filtered_tokens = filter_tokens(tokens, allowable_tokens)

token_freq = Counter(filtered_tokens)

# Output the top 10 most common allowable tokens
print("Top 10 Allowable Tokens:", token_freq.most_common(10))

# Close the connection
conn.close()